In [23]:
import dropbox
import xarray as xr
import io
import json

with open("/home/dlhogan/GitHub/pysumma/dropbox.json") as f:
    token = json.load(f)
    APP_KEY = token['APP_KEY']
    APP_SECRET = token['APP_SECRET']

def read_xarray_file_from_dropbox(dropbox_file_path, access_token):
    """
    Reads an xarray file directly from Dropbox into memory.

    Parameters:
        dropbox_file_path (str): Path to the file in Dropbox (e.g., '/folder/file.nc').
        access_token (str): Dropbox API access token.

    Returns:
        xarray.Dataset: Loaded dataset.
    """
    try:
        # Initialize Dropbox client
        dbx = dropbox.Dropbox(app_key=APP_KEY,
                              app_secret=APP_SECRET,
                              oauth2_refresh_token=access_token)
        
        # Download the file content into memory
        metadata, res = dbx.files_download(dropbox_file_path)
        file_content = io.BytesIO(res.content)
        
        # Load the file content as an xarray dataset
        dataset = xr.open_dataset(file_content)
        return dataset
    except Exception as e:
        raise RuntimeError(f"Failed to read file from Dropbox: {e}")


In [ ]:
import dropbox
import json
import requests
import webbrowser
import base64
with open("/home/dlhogan/GitHub/pysumma/dropbox.json") as f:
    token = json.load(f)
    APP_KEY = token['APP_KEY']
    APP_SECRET = token['APP_SECRET']
    ACCESS_CODE_GENERATED = token['ACCESS_CODE_GENERATED']
url = f"https://www.dropbox.com/oauth2/authorize?client_id={APP_KEY}&token_access_type=offline&response_type=code"

BASIC_AUTH = base64.b64encode(f'{APP_KEY}:{APP_SECRET}'.encode())

headers = {
    'Authorization': f"Basic {BASIC_AUTH}",
    'Content-Type': 'application/x-www-form-urlencoded',
}

data = f'code={ACCESS_CODE_GENERATED}&grant_type=authorization_code'

response = requests.post('https://api.dropboxapi.com/oauth2/token', data=data, auth=(APP_KEY, APP_SECRET), headers=headers)



In [24]:
# Example usage
# open .dropbox_token file and read the token
with open("/home/dlhogan/GitHub/pysumma/.dropbox_token") as f:
    token = f.read().strip()

access_token = token
site = "Stevens_Pass"
dropbox_file = f"/Apps/push-and-pull-pysumma/output/_{site}_timestep.nc"  # Path to the xarray file in Dropbox

ds = read_xarray_file_from_dropbox(dropbox_file, access_token)